In [2]:
import numpy as np

# Einführung

Das Modul [**Numpy**](https://numpy.org/doc/stable/index.html) bildet, so kann man sagen, die Grundlage des gesamten *Python Science Stack*. NumPy ist eine *numerische* Programmbibliothek für das Rechnen mit Vektoren und Matrizen. Darüber hinaus bietet NumPy effektive Array Datenstrukturen, sowie Funktionen für numerische Berechnungen.

Auch wenn Sie im täglichen Umgang mit NumPy eher weniger direkt in Berührung kommen, sondern mehr mit Pandas arbeiten werden (siehe Kapitel 4), so nutzt doch letzteres "unter der Haube" jenes NumPy. Mit NumPy ist es sehr leicht Rechenoperationen auf ganze Daten-Arrays anzuwenden, ohne *for-Schleifen* zu benutzen. Es lohnt sich also - in Vorbereitung auf den Umgang mit Pandas ein Grundverständnis von NumPy zu erwerben. Wenn Sie natürlich in ihrer wissenschaftlichen Praxis stärker auf numerische Berechnungen oder lineare Algebra setzen, dann sollten sie sich NumPy noch genauer ansehen.

NumPy ist die Grundlage des kompletten Python Science Stacks:

![Numpy Lifting](./img/numpy_lifting.png)


## Warum NumPy?

Es gibt vor allem zwei Gründe für die Entwicklung von NumPy, die übrigens schon im Jahre 1995 unter dem Acronym *Numeric* begann: einerseits eine effiziente Datenstruktur mit hoher Zugriffszeit, sowie die Möglichkeit des Rechnens mit diesen Datenstrukturen.

Werfen wir zuerst einen Blick auf das Thema Geschwindigkeit. Schreiben wir ein kleine Funktion, die einfach die ganzen Zahlen von 1 bis n summiert.

In [7]:
def sum_integers(n: int) -> int:
    s = 0
    i = 1
    while i < n:
        s += i
        i += 1
    return s

%timeit sum_integers(1_000_000)

69.5 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Anstatt einer "teuren" while-Schleife sollten wir vielleicht besser integrierte Python Funktionen wie die `sum()` Funktion oder *List-Comprehensions* verwenden, da diese auf angepassten C-Code basieren?

In [8]:
def sum_integers_py(n: int) -> int:
    return sum(range(n))

%timeit sum_integers_py(1_000_000)

14.5 ms ± 522 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Das sieht schon besser aus. Probieren wir das ganze noch mit NumPy aus.

In [9]:
def sum_integers_numpy(n: int) -> int:
    return np.array(np.arange(n), dtype=np.int32).sum()

%timeit sum_integers_numpy(1_000_000)

1.94 ms ± 85.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Wir sehen, dass NumPy die Ausführung von Code erheblich verbessern kann. Bei komplexeren Algorithmen und Berechnung kann NumPy die Ausführung sogar um den Faktor 100 verbessern.

Ein weiterer Punkt ist die einfache Anwendung von Rechenoperationen auf ganze Arrays, auch mehrdimensionale. Stellen wir uns eine zwei Vektoren mit Ganzzahlen vor, deren Elemente wir in Python paarweise addieren wollen. In reinem Python müsste man etwas wie folgt schreiben (ausserdem messen wir mal wieder die Zeit:

In [4]:
from typing import List
import random

def add_vectors(v1: List[int], v2: List[int]) -> List[int]:

    assert len(v1) == len(v2), "Beide Vektoren müssen gleich lang sein!"
    output = [v1[i] + v2[i] for i, _ in enumerate(v1)]

    return output

vec1 = [random.choices(range(1, 10), k=1_000_000)]
vec2 = [random.choices(range(1, 10), k=1_000_000)]

%timeit add_vectors(vec1, vec2)

4.52 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Mit NumPy ist der Code wesentlich simpler und auch schneller.

In [11]:
def add_vectors_np(np_arr1, np_arr2):
    return np_arr1 + np_arr2 # das ist alles!!

np_vec1 = np.random.randint(low=1, high=9, size=1_000_000)
np_vec2 = np.random.randint(low=1, high=9, size=1_000_000)

%timeit add_vectors_np(np_vec1, np_vec2)

1.24 ms ± 5.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Im Prinzip brauchen wir also nur den "+" Operator (Zeile 4), um zwei Vektoren elementweise zu addieren. Das geht, wie wir später noch sehen werden auch mit mehrdimensionalen Matrizen.

## NumPy Arrays: ndarray

Damit dies funktioniert, arbeitet NumPy mit sog. Arrays, Datenstrukturen, auf deren Elemente sehr schnell im Speicher zugegriffen werden kann. In Numpy heiẞen diese Array-Objekte **ndarrays** (N-dimensionale Arrays). Die NumPy Arrays unterscheiden sich von klassischen sequentiellen Datenstrukturen in Python wie Listen und Tupel in folgenden Punkten:
  - Bei ihrer Generierung haben *ndarrays* eine fest gesetzte Gröẞe, anders als Listen, die dynamisch wachsen können. Wir der Arrays verändert, wird ein neues Array erstellt und der alte im Speicher gelöscht.
  - Alle Elemente eines Arrays müssen den gleichen Datentyp haben und damit auch die gleiche Gröẞe im Speicher. Einzige Ausnahme: jedes Python Objekt kann auch Teil eines *ndarrays* sein.
  - Es sind, wie schon gezeigt, mathematische Funktionen für diese Arrays vorhanden, mit denen man diesen Objekten sehr effizient operieren kann, ohne viel Code zu schreiben.

Wie wird nun ein *ndarray* erzeugt? Hierfür gibt es die Funktion `np.array()`, dem wir eine ganze normale Python Liste samt Elementen übergeben.

In [34]:
obj = np.array([2, 3, 5])

print(obj)
print(obj.dtype)

[2 3 5]
int64


### Verschiedene Dimensionen

Auch mehrdimensionale Matrizen kann man so erzeugen. Dabei gibt dann die Methode `.shape` ein Tupel mit der Gröẞe der Dimensionen zurück.

In [3]:
obj1 = np.array([1, 2, 3, 4])

print(obj1.shape)

obj2 = np.array([[1, 2, 3 , 4], [4, 3, 2, 1]])

print(obj2.shape)

(4,)
(2, 4)


Den obigen Numpy Array `obj1` nennt man auch einen *Vektor*, da dieser nur *eine* Dimension hat. `obj2` nennt man eine *Matrix* (Pl. *Matrizen*), da es min. zwei Dimensionen hat.

Natürlich kann man dieses Dimensionen-Spielchen theoretisch unendlich weiterführen. Einen Array mit drei oder mehr Dimensionen nennt man *Tensor*.


Man erzeugt einen Tesor indem man einen `ndarray` aus einer Liste von zweidimensionalen Listen erzeugt. Einen dreidimensionalen Tensor kann man sich dabei wie einen Rubic's Cube vorstellen:

![Rubics Cube](./img/rubics_cube.png)


In [5]:
# Bau eines Tensors:

matrix1 = np.array([[1, 3, 4, 6], [3, 4, 6, 8]])
matrix2 = np.array([[3, 2, 4, 5], [7, 6, 8, 9]])
matrix3 = np.array([[0, 1, 9, 2], [2, 3, 4, 5]])

tensor = np.array([ matrix1, matrix2, matrix3 ])

print(tensor, "\n")

print(tensor.shape)

[[[1 3 4 6]
  [3 4 6 8]]

 [[3 2 4 5]
  [7 6 8 9]]

 [[0 1 9 2]
  [2 3 4 5]]] 

(3, 2, 4)


Mehr als drei Dimensionen mental zu visualisieren ist nicht leicht. Eine Analogie aus dem täglichen Leben mag aber helfen:

- **2d:** Zeilen und Spalten
- **3d:** Rubic's Cube
- **4d:** eine Schublade voll Rubic's Cubes
- **5d:** ein Schreibtisch mit mehreren Schubladen voll Rubic's Cubes
- **6d:** eine Halle voller Schreibtische voller ...
- **7d:** eine Strasse mit aneinandergereihten Hallen voll Schreibtischen voller ...
- **8d:** usw. usw.

## Vektoren und Matrizen direkt erzeugen lassen

Es gibt viele Methoden zur Erzeugung von Numpy Arrays. Einen Überblick über alle finden Sie [hier](https://numpy.org/doc/stable/reference/routines.array-creation.html).

Die wichtigsten folgen als Beispiel.

### np.zeros | np.ones | np.full

### np.linspace

### np.arange

### np.random

### Reshaping: Dimensionalität eines Arrays ändern

In [5]:
arr = np.random.random(100)

arr.shape = (5, 20)

arr

array([[0.15281761, 0.15964098, 0.08893567, 0.64866017, 0.0516616 ,
        0.96048058, 0.05741294, 0.19952948, 0.12818766, 0.81865567,
        0.46192452, 0.67290439, 0.66272931, 0.47101132, 0.50520642,
        0.75581837, 0.25842175, 0.9712677 , 0.59550128, 0.67523793],
       [0.99236063, 0.91764278, 0.12931028, 0.6868155 , 0.23384552,
        0.61017935, 0.54364076, 0.7509334 , 0.10911137, 0.31031499,
        0.97258971, 0.72684438, 0.61125331, 0.65826023, 0.11504543,
        0.97070358, 0.54193547, 0.04419186, 0.48600688, 0.3564139 ],
       [0.87751905, 0.06544486, 0.17557644, 0.31940278, 0.47164926,
        0.44479041, 0.72271459, 0.30613653, 0.70456346, 0.22075389,
        0.10068425, 0.81740623, 0.71518508, 0.95600626, 0.70650521,
        0.48807114, 0.94220705, 0.67608632, 0.80466476, 0.55197257],
       [0.21123925, 0.40780055, 0.00524915, 0.19442599, 0.76819553,
        0.39616808, 0.1746509 , 0.6983381 , 0.34763382, 0.19594432,
        0.68466007, 0.47105127, 0.74820837, 0

## Indexing und Slicing

Indexing funktioniert bei Numpy Arrays genauso mit den `[]` Operator wie in Python üblich.

In [6]:
my_arr = np.arange(10)

print(my_arr[0])

print(my_arr[-1])

0
9


In [8]:
my_arr.shape = (2, 5)

print(my_arr[0])

print(my_arr[0][-1])

[0 1 2 3 4]
4


Allerdings gibt es in Numpy eine effizientere Methode, um auf die Elemente mehrdimensionaler Arrays zu gehen. Man kann dem `[]` Operator auch ein Tupel an Zahlen übergeben:

```array[0][1] == array[(0, 1)]```

Dies generiert den *view* (was das ist werden wir gleich noch sehen) auf den Array in einem einzigen Schritt und nicht in zweien, wie im Beispiel. Und da sich im Data Science Bereich alles um Effizienz dreht, wollen wir auch so auf Elemente in Numpy Arrays zugreifen.

Nun gibt es aber noch "syntaktischen Zucker" für den Zugriff per Tupel: man kann die runden klammern einfach weg lassen.

```array[(0, 1)] == array[0, 1]```

In [10]:
print(my_arr[(0, -1)])

print(my_arr[0, -1])

4
4


## Numpy Datentypen dtypes

Das Typsystem von Numpy ist umfangreicher und angepasster als das Typsystem von Python.

### Typ Coersion

## Filtering

## Achsen

## Daten zusammenfügen und entfernen

### np.concatenate()

Mit der `NumPy` Methode `.concatenate` lassen sich mehrere Arrays zusammenfügen. Die Funktion nimmt ein Tupel, das die Arrays enthält, zusammen mit einem Argument für die Achse, an der zusammengefügt werden soll, entgegen:
```
np.concatenate((arr1, arr2, ...), axis=0)
```
Aber vorsicht. Die übergebenen Arrays müssen eine gleichlange Dimension haben, an der sie zusammengefügt werden können.

In [22]:
arr1 = np.array([[1, 2], [3, 4]])

arr2 = np.array([5, 6])

np.concatenate((arr1, arr2))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

Warum kommt hier eine Fehlermeldung? Schauen wir uns die Dimensionen der beiden Arrays einmal genauer an.

In [14]:
print(arr1.shape, arr2.shape)

(2, 2) (2,)


Wir sehen, dass `arr2` nur eine einzige Dimension hat, also ein Vektor ist. `arr1` dagegen ist eine Matrix mit 2 Dimensionen. Wir müssen also `arr2` erst in eine zweidimensionale Form mit Hilfe der Funktion `.reshape()` bringen. Dann klappt auch die Concatination.

In [26]:
arr2 = arr2.reshape((1, 2))

print(arr2.shape, '\n')

arr3 = np.concatenate((arr1, arr2))

print(arr3)

(1, 2) 

[[1 2]
 [3 4]
 [5 6]]


Natürlich kann man die Dimensionen auch anders setzen und über den Parameter `axis=` die Achse bennen, an der die zwei Arrays aneinandergefügt werden soll. Dazu müssen wir unseren Array `arr2` aber noch einmal in die richtige Form bringen.

In [28]:
arr2 = arr2.reshape((2, 1))

arr4 = np.concatenate((arr1, arr2), axis=1)

print(arr4)

[[1 2 5]
 [3 4 6]]


In [30]:
arr3 = arr3.reshape(2, 3)

arr5 = np.concatenate((arr2, arr3, arr4), axis=1)

print(arr5)

[[5 1 2 3 1 2 5]
 [6 4 5 6 3 4 6]]


### np.delete()

### np.split()

### np.stack()

## Numpy Arrays Laden und Speichern

### Dateistreams mit Python öffnen

Für das Lesen von Dateien von der Festplatte steht die Funktion `open()` in Python zur Verfügung. Als Arguemente braucht Sie mindestens einen Pfad zur Datei und den Dateinamen als String, sowie einen bestimmten *Modus* zum Öffnen der Datei ebenfalls als String. Optional können auch noch Informationen zur Codierung der Datei übergeben werden.

`open("Pfad zur Datei + Name", "Modus")`

Folgende Modi stehen zur Verfügung:

| Modus | Bedeutung                                  |
|-------|--------------------------------------------|
| "r"   | Datei wird nur lesend geöffnet.            |
| "w"   | Datei wird nur schreibend geöffnet.        |
| "r+"  | Datei wird lesend und schreibend geöffnet. |
| "rb+" | Datei wird im Binärformat geöffnet.        |

Damit kein Datenverlust erfolgt, muss eine Datei, wenn Sie eingelesen und verarbeitet wurde, **immer** mit der Methode `.close()` geschlossen werden!

In [5]:
file = open("img/numpy_logo.png", "r")

print(type(file))

file.close()

<class '_io.TextIOWrapper'>


Damit möglichst kein Datenverlust geschieht öffnet man Dateien in Python normalerweise in einem sog. *With-Block*. Dieser sorgt dafür, dass selbst bei missglückten Operationen auf den Dateistream, dieser wieder sauber geschlossen wird.

Die Syntax sieht wie folgt aus:
```
with open(...) as <name>:
    mach etwas mit <name>
```

In [7]:
with open("img/numpy_logo.png", "r") as file:
    print(type(file))

<class '_io.TextIOWrapper'>


### np.save() und np.load()

So können wir nun Numpy Arrays speichern und auch wieder öffnen. Dazu stehen und die Numpy Funktionen `np.save()` und `np.load()` zur Verfügung. Folgende *Dateiformate* stehen uns dabei zur Verfügung:

- `.csv` (Komma separierte Textdatei)
- `.txt` (reine Textdatei)
- `.pkl` (eine Pickle Datei)
- `.npy` (das NumPy eigene Format)

Am effizientesten ist tatsächlich die Speicherung als `.npy` Datei.

In [12]:
new_arr = np.random.random(100).reshape(5, 20)

with open("data/test_save.npy", "wb") as file:
    np.save(file, new_arr)

Genauso können wir mit der Funktion `np.load()` eine `.npy` Datei wieder einlesen.

In [13]:
with open("data/test_save.npy", "rb") as file:
    loaded_arr = np.load(file)
    
loaded_arr[1]

array([0.89129808, 0.19778473, 0.41641027, 0.67294669, 0.58948442,
       0.8583714 , 0.59618886, 0.80257135, 0.88723119, 0.29273734,
       0.44690206, 0.72438551, 0.92043573, 0.88708781, 0.80793182,
       0.30377051, 0.705016  , 0.82430651, 0.32786455, 0.67468352])

# Mathematische und Statistische Funktionen mit NumPy

## Mathematische Operationen

## Broadcasting

"Broadcasting" nennt man ...

Um erfolgreich einen Array auf einen anderen zu broadcasten, muss *eine von zwei Bedingungen* für **alle** Dimensionen der Arrays gelten:
- die Länge einer der zu vergleichende Dimensionen ist 1,
- oder beide zu vergleichenden Dimensionen sind gleich lang.

Um das besser zu verstehen, schauen wir uns ein Beispiel an:
```
Array 1: [[0, 0, 0, 0],   Array 2: [[0],
          [0, 0, 0, 0],             [0],
          [0, 0, 0, 0],             [0],
          [0, 0, 0, 0],             [0],
          [0, 0, 0, 0]]             [0]]
          
np.shape() von Array 1: (5, 4)
                        /    \
              gleich lang    eine Dim gleich 1
                        \    /
np.shape() von Array 2: (5, 1)
```
Ein Broadcasting von Array 1 und Array 2 funktioniert also, weil für jede der Dimensionen eine Bedingung erfüllt ist.

![Broadcasting von Matrizen](img/broadcasting_2.png)

In [3]:
# Achtung!

arr = np.arange(10).reshape(2, 5)
arr2 = np.array([0, 1])

arr + arr2

ValueError: operands could not be broadcast together with shapes (2,5) (2,) 

In [4]:
# die shapes passen nicht zusammen!

print(arr.shape, arr2.shape)

(2, 5) (2,)


In [8]:
# erst durch einen Reshape von arr2 geht es!

arr2 = arr2.reshape((2, 1))

print(arr + arr2)

[[ 0  1  2  3  4]
 [ 6  7  8  9 10]]


![Broadcasting nach Reshape](img/broadcasting_3.png)

## Aggregatsfunktionen

### np.sum()

Als Beispiel haben wir die Verkaufszahlen von 12 Monaten (Zeilen) von drei Industriezweigen.

In [39]:
monthly_sales_industries = np.array([[ 4134, 23925,  8657],
                                     [ 4116, 23875,  9142],
                                     [ 4673, 27197, 10645],
                                     [ 4580, 25637, 10456],
                                     [ 5109, 27995, 11299],
                                     [ 5011, 27419, 10625],
                                     [ 5245, 27305, 10630],
                                     [ 5270, 27760, 11550],
                                     [ 4680, 24988,  9762],
                                     [ 5312, 25405, 13401],
                                     [ 6630, 27797, 18403]])

Wenden wir auf diesen Array die Aggregatsfunktion `.sum()` an, so werden sämtliche Verkaufszahlen des Jahres summiert und als Wert zurückgegeben.

In [40]:
monthly_sales_industries.sum()

468633

Möchte man allerdings die Summen der einzelnen Industriezweige eines Jahres summieren, so muss man die Achse angeben, über die summiert werden soll. In diesem Fall müssten wir die Zeilen-Achse, also `axis=0` verwenden.

In [42]:
monthly_sales_industries.sum(axis=0)

array([ 54760, 289303, 124570])

Möchte man hingegen die Werte der einzelnen Monate aufsummieren, so muss man dementsprechend an der anderen Achse, also `axis=1` summieren.

In [44]:
monthly_sales_industries.sum(axis=1)

array([36716, 37133, 42515, 40673, 44403, 43055, 43180, 44580, 39430,
       44118, 52830])

Schade ist, dass sich hier nun aber die Shape des Arrays verändert hat. Da wir ja die einzelnen Spalten summiert haben, sollte die Ausgabe auch in Spaltenform erfolgen, um so vielleicht dieses "Summenspalte" mit dem Array für Verkaufszahlen zu konkatenieren.

Hierfür gibt es das Argument `keepdims`, dem man die boolschen Werte `True` und `False` übergeben kann.

In [45]:
monthly_sums = monthly_sales_industries.sum(axis=1, keepdims=True)

monthly_sums

array([[36716],
       [37133],
       [42515],
       [40673],
       [44403],
       [43055],
       [43180],
       [44580],
       [39430],
       [44118],
       [52830]])

In [48]:
np.concatenate((monthly_sales_industries, monthly_sums), axis=1)

array([[ 4134, 23925,  8657, 36716],
       [ 4116, 23875,  9142, 37133],
       [ 4673, 27197, 10645, 42515],
       [ 4580, 25637, 10456, 40673],
       [ 5109, 27995, 11299, 44403],
       [ 5011, 27419, 10625, 43055],
       [ 5245, 27305, 10630, 43180],
       [ 5270, 27760, 11550, 44580],
       [ 4680, 24988,  9762, 39430],
       [ 5312, 25405, 13401, 44118],
       [ 6630, 27797, 18403, 52830]])



## Maße der zentralen Tendenz

## Maße der Streuung 

### Quantile / Percentile: np.quantile

Mit `np.quantile` lassen sich die Quantile einer Stichprobe berechnen. `np.quantile` arbeitet dabei, dass man als Argumente ein Array, sowie die Liste der Quantile zwischen 0 und 1, die man berechnen will.

In [12]:
# wir berechnen die Quartile 25%, 50% und 75%

koerpergroessen = np.random.normal(1.75, .15, 100) # 100 zufällig normalverteilte Körpergröẞen

np.quantile(koerpergroessen, [.25, .5, .75])

array([1.66539975, 1.76501605, 1.88188364])

Wollen wir mehr Quantile, ist es mühselig, diese als Liste auszuschreiben. Der Einfachheit halber können wir die Funktion `np.linspace` benutzen, die wir schon kennengelernt haben und die eine kontinuierliche Zahlenreihe in gleichgroẞe Stücke "zerschneidet". So können wir die Zahlenreihe von 0 bis 1 benutzen, um diese in Quantile zu zerschneiden und an `np.quantile` zu übergeben.

Hier ein paar Beispiele:

In [13]:
# Quintile (5 Quantile):

print("Quintile:", np.quantile(koerpergroessen, np.linspace(0, 1, 5)))

# Decile (10 Quantile):

print("Centile:", np.quantile(koerpergroessen, np.linspace(0, 1, 10)))

Quintile: [1.4515198  1.66539975 1.76501605 1.88188364 2.08062421]
Centile: [1.4515198  1.59257309 1.66420276 1.68446388 1.74643079 1.78222782
 1.8361634  1.88969072 1.98373368 2.08062421]


# Wahrscheinlichkeitstheorie